In [4]:
!pip install pandas networkx matplotlib spacy scipy
!pip install -U spacy
!python -m spacy download pt_core_news_lg

Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 148, in _get_module_details
  File "<frozen runpy>", line 112, in _get_module_details
  File "c:\Users\loli1\Documents\AnotomiaDoOdio\.venv\Lib\site-packages\spacy\__init__.py", line 6, in <module>
    from .errors import setup_default_warnings
  File "c:\Users\loli1\Documents\AnotomiaDoOdio\.venv\Lib\site-packages\spacy\errors.py", line 3, in <module>
    from .compat import Literal
  File "c:\Users\loli1\Documents\AnotomiaDoOdio\.venv\Lib\site-packages\spacy\compat.py", line 39, in <module>
    from thinc.api import Optimizer  # noqa: F401
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\loli1\Documents\AnotomiaDoOdio\.venv\Lib\site-packages\thinc\api.py", line 1, in <module>
    from .backends import (
  File "c:\Users\loli1\Documents\AnotomiaDoOdio\.venv\Lib\site-packages\thinc\backends\__init__.py", line 17, in <module>
    from .cupy_ops import CupyOps
  

In [2]:
import pandas as pd              # Para manipular os dados (CSVs, DataFrames)
import re                        # Para usar expressões regulares na limpeza do texto (remover @)
import networkx as nx            # Para criar e manipular os grafos
import matplotlib.pyplot as plt  # Para visualizar os grafos
import spacy                     # A biblioteca principal para processamento de linguagem em português
from itertools import combinations
import spacy
from scipy.io import arff
import os

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Carregar o modelo de linguagem em português do spaCy
# Isso substitui os downloads do NLTK
try:
    nlp = spacy.load('pt_core_news_sm')
    print("Bibliotecas essenciais e modelo 'pt_core_news_sm' do spaCy carregados com sucesso!")
except OSError:
    print("Modelo 'pt_core_news_sm' não encontrado. Por favor, execute a célula de instalação:")
    print("!python -m spacy download pt_core_news_sm")

# Importar o spaCy e carregar o modelo em português
nlp = spacy.load('pt_core_news_sm')
print("Modelo 'pt_core_news_sm' do spaCy carregado.")

In [ ]:
# Defina o nome dos seus arquivos de entrada
hateBR = './data/01-raw/hateBr/HateBRXplain.csv'
offComBR = './data/01-raw/offComBR/OffComBR3.arff'
olidBR = './data/01-raw/test-00000-of-00001-914dbee7561d2266.parquet'
fortuna = './data/01-raw/fortuna/2019-05-28_portuguese_hate_speech_binary_classification.csv'
toLDBR = './data/01-raw/toLDRBR/ToLD-BR.csv'
tuPyE = './data/01-raw/tuPyE/binary_test.csv'

In [ ]:
dados, meta_info = arff.loadarff(caminho_arff)
    
# Converte os dados carregados para um DataFrame do pandas
df_arff = pd.DataFrame(dados)

# ARQUIVOS ARFFgeralmente carregam texto como "bytes" (ex: b'texto').
# Precisamos decodificá-los para strings normais (utf-8).
# Esta linha seleciona todas as colunas de texto e as decodifica.
colunas_texto = df_arff.select_dtypes(include=['object']).columns
df_arff[colunas_texto] = df_arff[colunas_texto].apply(lambda x: x.str.decode('utf-8'))

print("--- Pré-visualização dos dados do .arff ---")
print(df_arff.head())

# Salva o DataFrame como .csv
df_arff.to_csv(caminho_csv_saida_1, index=False, encoding='utf-8')

In [ ]:
def pre_processar_pt(caminho_csv: str, nome_coluna: str) -> pd.DataFrame:
    """
    Carrega um CSV, remove duplicatas e aplica limpeza e lematização otimizada
    para o português em uma coluna de texto.
    """
    # 1. Carregar o dataset
    df = pd.read_csv(caminho_csv)
    print(f"Dataset original carregado com {len(df)} linhas.")

    # 2. Remover duplicatas
    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)
    print(f"Dataset após remoção de duplicatas possui {len(df)} linhas.")

    # 3. Limpeza e Lematização com spaCy
    # Criar uma nova coluna para o texto processado
    df['texto_processado'] = df[nome_coluna].apply(lambda texto: limpar_e_lematizar_spacy(texto))

    return df

def limpar_e_lematizar_spacy(texto: str) -> str:
    """
    Aplica limpeza e lematização de alta qualidade para o português usando spaCy.
    """
    # Remover menções (@usuario)
    texto = re.sub(r'@\w+', '', texto)
    # Remover hashtags (#palavra) mas manter a palavra
    texto = re.sub(r'#', '', texto)
    # Converter para minúsculas
    texto = texto.lower()

    # Processar o texto com o modelo spaCy
    doc = nlp(texto)

    # Obter os lemas, ignorando pontuação e stopwords (palavras comuns como 'e', 'a', 'o')
    # Isso torna o grafo mais limpo e significativo
    lemmas = [
        token.lemma_ for token in doc 
        if not token.is_punct and not token.is_space and not token.is_stop
    ]
    
    # Exemplo: A palavra "eram" se torna "ser", "fizemos" se torna "fazer".

    return " ".join(lemmas)

# A função de montar o grafo permanece a mesma
def montar_grafo_de_coocorrencia(series_texto: pd.Series) -> nx.Graph:
    """
    Cria um grafo NetworkX a partir de uma série de textos processados.
    Nós = palavras (lemas)
    Arestas = coocorrência de palavras na mesma sentença (tweet/comentário)
    """
    G = nx.Graph()
    
    for texto in series_texto:
        palavras = texto.split()
        
        # Adiciona nós (evita duplicatas automaticamente)
        for palavra in palavras:
            G.add_node(palavra)
            
        # Adiciona arestas baseadas nas combinações de palavras no mesmo texto
        for w1, w2 in combinations(palavras, 2):
            if G.has_edge(w1, w2):
                if 'weight' in G[w1][w2]:
                    G[w1][w2]['weight'] += 1
                else:
                     G[w1][w2]['weight'] = 2
            else:
                G.add_edge(w1, w2, weight=1)
                
    return G